# **LAB 02: LÀM VIỆC VÀ TRỰC QUAN HÓA DỮ LIỆU CHUỖI THỜI GIAN**

#### Môn học: Trực quan hóa dữ liệu

#### Nhóm: 5


## **A. Thu thập dữ liệu**




nhóm 5 lựa chọn bộ dữ liệu *U.S. Air Pollution* cho lab 01. Bộ dữ liệu được chia sẻ miễn phí trên nền tảng data.world. (Link dataset: https://data.world/data-society/us-air-pollution-data)

License: <a href="http://opendatacommons.org/licenses/dbcl/1.0/">Database: Open Database, Contents: Database Contents</a>


Bộ dữ liệu được thu thập bằng cách scraped từ database của trang U.S. EPA : https://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/download_files.html


## **B. Khám phá dữ liệu**

### **Import các thư viện**

In [106]:
#!pip install pingouin
#!pip install category_encoders
#!pip install plotly
#!pip install yellowbrick

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
"""
import seaborn as sns
import plotly.express as px
import plotly.offline as py
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import pingouin as pg #for ANOVA

#from sklearn.preprocessing import OneHotEncoder
from sklearn import decomposition
import category_encoders as ce
from sklearn.cluster import KMeans
#from sklearn.metrics import silhouette_score
from scipy.stats import zscore
from yellowbrick.cluster import KElbowVisualizer
"""

%matplotlib inline  

### **Đọc file dữ liệu vào dataframe**

In [108]:
df=pd.read_csv("pollution_us_2000_2016.csv")
df.head()


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Units,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,49.0,19,46,Parts per million,0.022500,0.040,10,34,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,49.0,19,46,Parts per million,0.022500,0.040,10,34,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,49.0,19,46,Parts per million,0.022500,0.040,10,34,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,49.0,19,46,Parts per million,0.022500,0.040,10,34,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,22.958333,36.0,19,34,Parts per million,0.013375,0.032,10,27,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


### **Tiền xử lý và khám phá dữ liệu**

Đầu tiên ta sẽ xóa dòng index của dataset.

In [109]:
df=df.drop(columns="Unnamed: 0")

**Số dòng** và **số cột** của bộ dữ liệu: 2000 dòng, 8 cột

In [110]:
num_rows, num_cols = df.shape
print("Num rows: ", num_rows)
print("Num cols: ", num_cols)

Num rows:  1746661
Num cols:  28


**Ý nghĩa của mỗi dòng**: Thông tin về nồng độ NO2, O3, SO2, CO trong không khí ở một thành phố trong một ngày.

In [111]:
pd.set_option('display.max_columns', None)
df.head(1)

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Units,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,49.0,19,46,Parts per million,0.0225,0.04,10,34,Parts per billion,3.0,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN


Kiểm tra xem có dòng nào bị trùng lặp hay không:

In [112]:
df.duplicated().sum()

5032

Thực hiện loại bỏ các dòng trùng lặp:

In [113]:
df=df.drop_duplicates()

**Ý nghĩa của mỗi cột**: 

- `State Code` : Mã tiểu bang.
- `County code` : Mã quận.
- `Site Num` : Số site ở mỗi quận.
- `Address`: Địa chỉ
- `State` : Tên tiểu bang.
- `County` : Tên quận
- `City` : Tên thành phố
- `Date Local` : Ngày thực hiện đo.


4 chất gây ô nhiễm không khí ($NO_2, O_3, SO_2$ và  $O_3$), ứng với mỗi chất có 5 cột giá trị. Ví dụ đối với $NO_2$:

- `NO2 Units` : đơn vị đo lường NO2
- `NO2 Mean` : trung bình nồng độ NO2 trong một ngày cụ thể.
- `NO2 AQI` : chỉ số chất lượng không khí được tính toán bởi NO2 trong một ngày cụ thể.
-   `NO2 1st Max Value` : Giá trị lớn nhất của nồng độ NO2 trong ngày.
- `NO2 1st Max Hour` : Thời gian mà nồng độ NO2 đạt giá trị lớn nhất.

Ta sẽ thực hiện chuẩn hóa tên các cột để thuận tiện hơn cho việc sử dụng về sau:

In [114]:
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

**Kiểu dữ liệu của mỗi cột:**

In [115]:
df.dtypes

State_Code             int64
County_Code            int64
Site_Num               int64
Address               object
State                 object
County                object
City                  object
Date_Local            object
NO2_Units             object
NO2_Mean             float64
NO2_1st_Max_Value    float64
NO2_1st_Max_Hour       int64
NO2_AQI                int64
O3_Units              object
O3_Mean              float64
O3_1st_Max_Value     float64
O3_1st_Max_Hour        int64
O3_AQI                 int64
SO2_Units             object
SO2_Mean             float64
SO2_1st_Max_Value    float64
SO2_1st_Max_Hour       int64
SO2_AQI              float64
CO_Units              object
CO_Mean              float64
CO_1st_Max_Value     float64
CO_1st_Max_Hour        int64
CO_AQI               float64
dtype: object

In [116]:
datime_df = pd.DataFrame(df['Date_Local'].copy())

Cột `Date_Local` chưa có kiểu dữ liệu phù hợp, ta sẽ thực hiện chuyển kiểu dữ liệu về kiểm **datetime**

In [117]:
df['Date_Local'] = pd.to_datetime(df['Date_Local'], format='%Y-%m-%d')

Chuyển các cột `State_Code`, `County_Code` sang **string**

In [118]:
df[['State_Code', 'County_Code']]=df[['State_Code', 'County_Code']].astype(str)

**Kiểm tra giá trị bị thiếu:**

In [119]:
df.isna().sum()/df.shape[0]

State_Code           0.00000
County_Code          0.00000
Site_Num             0.00000
Address              0.00000
State                0.00000
County               0.00000
City                 0.00000
Date_Local           0.00000
NO2_Units            0.00000
NO2_Mean             0.00000
NO2_1st_Max_Value    0.00000
NO2_1st_Max_Hour     0.00000
NO2_AQI              0.00000
O3_Units             0.00000
O3_Mean              0.00000
O3_1st_Max_Value     0.00000
O3_1st_Max_Hour      0.00000
O3_AQI               0.00000
SO2_Units            0.00000
SO2_Mean             0.00000
SO2_1st_Max_Value    0.00000
SO2_1st_Max_Hour     0.00000
SO2_AQI              0.49973
CO_Units             0.00000
CO_Mean              0.00000
CO_1st_Max_Value     0.00000
CO_1st_Max_Hour      0.00000
CO_AQI               0.50010
dtype: float64

Có gần 50% giá trị bị thiếu ở 2 cột `SO2_AQI` và `CO_AQI`

**Phân bố giá trị của các cột numerical:**

Bảng thống kê cơ bản về các phân bố giá trị của các thuộc tính **numeric**:

In [120]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()


In [121]:
df[numeric_cols].describe().round(2)

,Site_Num,NO2_Mean,NO2_1st_Max_Value,NO2_1st_Max_Hour,NO2_AQI,O3_Mean,O3_1st_Max_Value,O3_1st_Max_Hour,O3_AQI,SO2_Mean,SO2_1st_Max_Value,SO2_1st_Max_Hour,SO2_AQI,CO_Mean,CO_1st_Max_Value,CO_1st_Max_Hour,CO_AQI
count,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,1741629.00,871285.00,1741629.00,1741629.00,1741629.00,870641.00
mean,1120.18,12.82,25.41,11.73,23.89,0.03,0.04,10.17,36.04,1.87,4.50,9.66,7.12,0.37,0.62,7.87,6.00
std,2004.00,9.51,16.00,7.88,15.16,0.01,0.02,4.00,19.74,2.76,7.69,6.73,11.94,0.31,0.64,7.98,5.85
min,1.00,-2.00,-2.00,0.00,0.00,0.00,0.00,0.00,0.00,-2.00,-2.00,0.00,0.00,-0.44,-0.40,0.00,0.00
25%,9.00,5.75,13.00,5.00,12.00,0.02,0.03,9.00,25.00,0.26,0.90,5.00,1.00,0.18,0.29,0.00,2.00
50%,60.00,10.74,24.00,9.00,23.00,0.03,0.04,10.00,33.00,0.99,2.00,8.00,3.00,0.29,0.40,6.00,5.00
75%,1039.00,17.71,35.60,20.00,33.00,0.03,0.05,11.00,42.00,2.33,5.00,14.00,9.00,0.47,0.80,13.00,8.00
max,9997.00,139.54,267.00,23.00,132.00,0.10,0.14,23.00,218.00,321.62,351.00,23.00,200.00,7.51,19.90,23.00,201.00


**Phân bố giá trị của các cột categorical:**

In [122]:
df[set(df.columns)-set(numeric_cols)].describe()

,Date_Local,State,State_Code,Address,SO2_Units,County,O3_Units,City,NO2_Units,CO_Units,County_Code
count,1741629,1741629,1741629,1741629,1741629,1741629,1741629,1741629,1741629,1741629,1741629
unique,5996,47,47,204,1,133,1,144,1,1,73
top,2013-04-17 00:00:00,California,6,PIKE AVE AT RIVER ROAD,Parts per billion,Los Angeles,Parts per million,Not in a city,Parts per billion,Parts per million,13
freq,408,575628,575628,35117,1741629,93371,1741629,138375,1741629,1741629,149178
first,2000-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2016-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **C. Khám phá mối quan hệ trong dữ liệu**

##### **Trực quan dạng Map về diễn biến: `CO_Mean` trong các `State` qua các `mùa trong năm`**

Khí CO (carbon monoxide) chủ yếu là các hợp chất carbon không hoàn toàn được đốt cháy trong môi trường có ít oxi. Được thải ra từ nhiều nguồn như: giao thông, công nghiệp, đốt cháy rừng.

Hoa Kỳ là một trong những quốc gia có mức độ ô nhiễm CO cao. Tác động của khí CO đối với môi trường và sức khỏe con người có thể được nhìn nhận như sau:

- Môi trường: CO có tác động tiêu cực đến chất lượng không khí. Nó góp phần vào sự hình thành ô nhiễm không khí, đặc biệt là trong các khu vực đô thị và các khu vực có lưu lượng giao thông lớn.

- Sức khỏe con người: Hít thở khí CO gây ra nhiều vấn đề sức khỏe. Khi tiếp xúc với CO trong thời gian dài, người ta có thể gặp các vấn đề như đau đầu, mệt mỏi, buồn nôn, khó thở và thậm chí gây ra tử vong. Đối với những người già, trẻ em và những người bị bệnh tim, CO có thể gây ra những vấn đề sức khỏe nghiêm trọng hơn.

In [123]:
from plotly.offline import  iplot

In [124]:
CO_df = df[['Address','State','County','City','CO_Units','CO_Mean','CO_1st_Max_Value','CO_1st_Max_Hour','CO_AQI']]
CO_df['Date_Local'] = datime_df
CO_df.head()

,Address,State,County,City,CO_Units,CO_Mean,CO_1st_Max_Value,CO_1st_Max_Hour,CO_AQI,Date_Local
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,Parts per million,1.145833,4.2,21,NaN,2000-01-01
1,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,Parts per million,0.878947,2.2,23,25.0,2000-01-01
2,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,Parts per million,1.145833,4.2,21,NaN,2000-01-01
3,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,Parts per million,0.878947,2.2,23,25.0,2000-01-01
4,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,Parts per million,0.850000,1.6,23,NaN,2000-01-02


In [125]:
CO_df = CO_df.groupby(['Address','State','County','City','Date_Local']).mean().reset_index()
CO_df.head()

,Address,State,County,City,Date_Local,CO_Mean,CO_1st_Max_Value,CO_1st_Max_Hour,CO_AQI
0,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-01,1.244518,1.50,2.5,16.0
1,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-02,1.054167,1.25,0.0,14.0
2,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-03,1.047916,1.20,15.0,14.0
3,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-04,0.927083,1.05,0.0,13.0
4,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-05,0.764584,0.95,21.5,10.0


In [126]:
codf = CO_df[CO_df['State']!='Country Of Mexico']
# get our total number
totalTuples = codf.count()['State']
#totalTuples
# add year and month columns
tempYear = []
tempMonth = []
for i in range(totalTuples):
    delement = (codf['Date_Local'].iloc[i]).split('-')
    tempYear.append(int(delement[0]))
    tempMonth.append(delement[0]+'-'+delement[1])
codf['Year'] = tempYear
codf['Month'] = tempMonth
codf.head()

,Address,State,County,City,Date_Local,CO_Mean,CO_1st_Max_Value,CO_1st_Max_Hour,CO_AQI,Year,Month
0,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-01,1.244518,1.50,2.5,16.0,2000,2000-01
1,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-02,1.054167,1.25,0.0,14.0,2000,2000-01
2,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-03,1.047916,1.20,15.0,14.0,2000,2000-01
3,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-04,0.927083,1.05,0.0,13.0,2000,2000-01
4,6100 ARLINGTON BLVD MONTG WARD,Virginia,Fairfax,Seven Corners,2000-01-05,0.764584,0.95,21.5,10.0,2000,2000-01


In [127]:
mapData = codf[(codf['Month'] >= '2014-12') & (codf['Month'] <= '2015-11')]


winterdf = mapData[mapData['Month'].isin(['2014-12', '2015-01', '2015-02'])].groupby('State').mean().reset_index().sort_values('State')
springdf = mapData[mapData['Month'].isin(['2015-03', '2015-04', '2015-05'])].groupby('State').mean().reset_index().sort_values('State')
summerdf = mapData[mapData['Month'].isin(['2015-06', '2015-07', '2015-08'])].groupby('State').mean().reset_index().sort_values('State')
autumndf = mapData[mapData['Month'].isin(['2015-09', '2015-10', '2015-11'])].groupby('State').mean().reset_index().sort_values('State')

abbState = ['US State:', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'Commonwealth/Territory:', 'American Samoa', 'District Of Columbia', 'Federated States of Micronesia', 'Guam', 'Marshall Islands', 'Northern Mariana Islands', 'Palau', 'Puerto Rico', 'Virgin Islands', 'Military "State":', 'Armed Forces Africa', 'Armed Forces Americas', 'Armed Forces Canada', 'Armed Forces Europe', 'Armed Forces Middle East', 'Armed Forces Pacific']
abbAB = ['Abbreviation:', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'Abbreviation:', 'AS', 'DC', 'FM', 'GU', 'MH', 'MP', 'PW', 'PR', 'VI', 'Abbreviation:', 'AE', 'AA', 'AE', 'AE', 'AE', 'AP']
abbDF = pd.DataFrame([abbState,abbAB]).transpose()

#small correction, so things go smooth ahead
abbDF.iloc[53][0] = 'District Of Columbia'

def CO_Mean_Season(df):
    mapA = []
    mapS = []
    for i in df.index:
        mapA.append(str(df['CO_Mean'].iloc[i])[:5]+' ppm')
        mapS.append(abbDF[abbDF[0] == df['State'].iloc[i]][1].values[0])
    df['text'] = mapA
    df['code'] = mapS

CO_Mean_Season(winterdf)
CO_Mean_Season(springdf)
CO_Mean_Season(summerdf)
CO_Mean_Season(autumndf)

**Map state-wise distribution of CO Mean in spring**

In [128]:
data = dict(type='choropleth',
            locations = springdf['code'],
            z = springdf['CO_Mean'],
            locationmode = 'USA-states',
            text = springdf['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"CO Mean in ppm"}
            ) 
layout = dict(title = 'CO Mean Value in Spring by State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

Dựa trên dữ liệu CO_Mean vào mùa xuân của các bang Hoa Kỳ, chúng ta có thể đưa ra một số nhận xét sau:

- Các giá trị trung bình CO_Mean vào mùa xuân của các bang có sự biến động đáng kể, từ khoảng 0.033 ppm ở `Wyoming` đến khoảng 0.504 ppm ở `Florida`. Sự khác biệt này cho thấy mức độ ô nhiễm không khí có thể thay đổi đáng kể giữa các bang trong mùa xuân.

- Có một số bang có mức độ ô nhiễm CO trong mùa xuân cao hơn so với các bang khác. Các bang có CO_Mean cao nhất vào mùa xuân là `Florida` (0.504 ppm), `Hawaii` (0.356 ppm), và `Colorado` (0.340 ppm). Điều này có thể liên quan đến các yếu tố như công nghiệp, giao thông vận tải và môi trường sống trong từng bang.

- Có các bang khác nhau có mức độ ô nhiễm CO trong mùa xuân ở mức trung bình. 

- Các bang có CO_Mean thấp nhất vào mùa xuân là `Wyoming` (0.033 ppm), `New Hampshire` (0.185 ppm), và `North Dakota` (0.182 ppm). Điều này cho thấy rằng các bang này có mức độ ô nhiễm không khí thấp hơn trong mùa xuân so với các bang khác.

**Map state-wise distribution of CO Mean in summer**

In [129]:
data = dict(type='choropleth',
            locations = summerdf['code'],
            z = summerdf['CO_Mean'],
            locationmode = 'USA-states',
            text = summerdf['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"CO Mean in ppm"}
            ) 
layout = dict(title = 'CO Mean Value in Summer by State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)


Dựa vào dữ liệu CO_Mean vào mùa hè tại các bang Hoa Kỳ, ta có thể nhận xét một số điểm quan trọng như sau:

- Mức độ ô nhiễm CO: Các giá trị CO_Mean trong mùa hè của các bang Hoa Kỳ có sự biến đổi đáng kể. Từ dữ liệu, ta thấy rằng:
    - Các bang `Florida` (0.510 ppm), `Alaska` (0.283 ppm) và `Colorado` (0.338 ppm) có mức ô nhiễm CO cao nhất trong mùa hè. 
    - Trong khi đó, `Wyoming` (0.053 ppm), `Pennsylvania` (0.144 ppm) và `Texas	` (0.151 ppm) có mức ô nhiễm CO thấp nhất.

- Ảnh hưởng địa phương: Như đã đề cập ở trên, các yếu tố như công nghiệp, giao thông, môi trường tự nhiên và chính sách bảo vệ môi trường có thể ảnh hưởng đến mức độ ô nhiễm CO của mỗi bang. Ví dụ, các bang có dân số đông đúc và hoạt động công nghiệp mạnh như `Florida` và `Alaska` có mức độ ô nhiễm CO cao hơn so với các bang như `Wyoming` và `Pennsylvania`.

**Map state-wise distribution of CO Mean in autumn**

In [130]:
data = dict(type='choropleth',
            locations = autumndf['code'],
            z = autumndf['CO_Mean'],
            locationmode = 'USA-states',
            text = autumndf['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"CO Mean in ppm"}
            ) 
layout = dict(title = 'CO Mean Value in Autumn by State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)


Dựa trên dữ liệu CO_Mean của mùa thu ở các bang tại Hoa Kỳ, ta có thể đưa ra nhận xét sau:

- Bang `Florida` có giá trị CO_Mean cao nhất vào mùa thu, đạt 0.509 ppm. Điều này cho thấy mức độ ô nhiễm CO ở bang này khá cao trong mùa thu.

- Các bang `Illinois`, `Hawaii` và `Colorado` cũng có giá trị CO_Mean vào mùa thu khá cao, với khoảng từ 0.326 ppm đến 0.365 ppm. Điều này cho thấy mức độ ô nhiễm CO đáng kể trong mùa thu ở các bang này.
 
- Các bang `Pennsylvania`, `Texas` và `Wyoming` có giá trị CO_Mean vào mùa thu thấp nhất, chỉ khoảng từ 0.0523 ppm đến 0.151 ppm. Điều này cho thấy mức độ ô nhiễm CO trong mùa thu ở các bang này thường thấp.

Các bang khác như `California`, `Ohio` và `Virginia` có giá trị CO_Mean vào mùa thu từ 0.224 ppm đến 0.266 ppm, ở mức trung bình so với các bang khác.

**Map state-wise distribution of CO Mean in winter**

In [131]:
data = dict(type='choropleth',
            locations = winterdf['code'],
            z = winterdf['CO_Mean'],
            locationmode = 'USA-states',
            text = winterdf['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"CO Mean in ppm"}
            ) 
layout = dict(title = 'CO Mean Value in Winter by State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

Dựa trên dữ liệu CO_Mean vào mùa đông của các bang Hoa Kỳ, ta có thể đưa ra một số giải thích cho kết quả này:

- `Alaska` (AK): Alaska có mức trung bình CO_Mean vào mùa đông cao nhất đạt 0.755480 ppm trong các bang (cao nhất khi ghi nhận vào các mùa). Điều này có thể được giải thích bởi sự phụ thuộc vào hệ thống sưởi và gia tăng nhiệt, do Alaska có khí hậu lạnh và nhu cầu sưởi ấm cao. Việc đốt nhiên liệu như dầu, củi và than có thể góp phần vào mức CO cao.

- `Utah` (UT), `Arizona` (AZ), `Colorado` (CO), `Florida` (Fl) : Utah (0.551 ppm), Arizona (0.556 ppm), Colorado (0.517 ppm) và Florida (0.478 ppm) cũng có mức trung bình CO cao đáng chú ý. Cả hai bang này có địa hình núi non phức tạp và hẹp hơn, dẫn đến tích tụ không khí ô nhiễm và khói trong các thung lũng và khe núi. Ngoài ra, việc sử dụng hệ thống sưởi và gia tăng nhiệt trong những ngày lạnh cũng có thể đóng góp vào mức CO cao.

- `California` (CA): California, mặc dù có mức trung bình CO_Mean không quá cao, nhưng vẫn ở mức trung bình. California có dân số đông đúc và giao thông ô tô phức tạp, dẫn đến lưu lượng phương tiện cơ giới lớn và khí thải CO từ động cơ ô tô. Điều này có thể là một yếu tố đóng góp vào mức CO trung bình trong bang này.

- `Wyoming` (WY) và `North Dakota` (ND): Wyoming và North Dakota có mức trung bình CO_Mean thấp nhất trong các bang. Cả hai bang này có dân số thưa thớt và ít công nghiệp, dẫn đến mức ô nhiễm không khí thấp hơn. Ngoài ra, yếu tố địa lý và khí hậu cũng có thể đóng vai trò, với không gian rộng lớn và gió mạnh giúp giảm tích tụ khí CO.

Nhìn chung, lượng khí CO thải ra được ghi nhận tại các bang vào mùa đông tăng mạng so với các mùa khác trong năm. Nguyên nhân chính có thể đến từ hoạt động đốt than, củi sưởi ấm cho mùa đông của người dân.

#### **Nhận xét chung**

Dựa vào nhận xét từ dữ liệu CO_Mean qua các mùa, có một số bang của Hoa Kỳ thường có lượng CO_Mean thải ra luôn thuộc top cao nhất và thấp nhất qua các mùa. Đây có thể được giải thích bởi một số yếu tố sau đây:

- `Florida`: Florida có mức CO_Mean cao nhất qua các mùa. Lý do có thể là do tình hình giao thông và đô thị phát triển tại bang này. Florida là một trong những điểm đến du lịch phổ biến, thu hút lượng lớn khách du lịch và dân số tăng cao. Sự tăng cường hoạt động giao thông và công nghiệp có thể góp phần vào việc thải ra lượng CO cao.

- `Alaska` và `Colorado`: Alaska và Colorado cũng có mức CO_Mean cao nhất qua các mùa. Các yếu tố như khí hậu, địa hình, và hoạt động công nghiệp tại các bang này có thể đóng vai trò quan trọng. Alaska có diện tích rộng lớn, phong cảnh núi non và đông dân cư thưa thớt. Việc sử dụng nhiên liệu hóa thạch cho việc sưởi ấm và vận chuyển có thể tạo ra lượng CO lớn. Colorado có một ngành công nghiệp khí hóa lớn và đô thị hóa nhanh, dẫn đến tăng cường hoạt động giao thông và khói bụi từ công trình xây dựng, góp phần vào mức độ ô nhiễm CO cao.

- `Wyoming`, `Pennsylvania`, `Texas`: Các bang này có mức CO_Mean thấp nhất qua các mùa. Có thể lý giải bởi sự thưa thớt dân số, ít hoạt động công nghiệp, và diện tích rừng/phong cảnh tự nhiên phong phú. Wyoming, với diện tích rừng rộng lớn, có ít hoạt động công nghiệp so với một số bang khác. Pennsylvania và Texas, mặc dù có một số thành phố lớn, cũng có nhiều vùng nông thôn và không có sự phát triển công nghiệp mạnh mẽ trong một số khu vực, góp phần vào mức độ ô nhiễm CO thấp.

Hoa Kỳ đã thực hiện nhiều biện pháp để giảm tác động của khí CO. Điều này bao gồm việc áp dụng các tiêu chuẩn nghiêm ngặt về khí thải xe cộ, kiểm soát quá trình sản xuất công nghiệp và sử dụng các công nghệ sạch hơn để giảm khí thải. Ngoài ra, việc khuyến khích sử dụng phương tiện giao thông công cộng và các biện pháp khuyến khích đi bộ, xe đạp cũng có thể giúp giảm lượng CO thải ra.

## **D. Xây dựng mô hình học máy:**

### **1. Chuẩn bị dữ liệu**: 

### **2. Xây dựng mô hình**

### **3. Trực quan hóa mô hình:**

## **E. Tài liệu tham khảo**

https://simerp.io/blog/tim-hieu-nhu-cau-khach-hang/

https://vitalflux.com/pca-explained-variance-concept-python-example/

https://www.datacamp.com/tutorial/decision-tree-classification-python

https://www.kaggle.com/code/utkarshsaxenadn/shop-customer-clustering?fbclid=IwAR29tq6_MlfU848Svyr_Rghg9U3nxiIYaZa3kHgURPZvRh1NwN8Lo8wrkp0